In [ ]:
from llama_cpp import Llama

# Set gpu_layers to the number of layers to offload to GPU. Set to 0 if no GPU acceleration is available on your system.
llm = Llama(
  model_path="./saiga_mistral_7b.Q4_K_M.gguf",  # Download the model file first
  n_ctx=8192,  # The max sequence length to use - note that longer sequence lengths require much more resources
  n_threads=8,            # The number of CPU threads to use, tailor to your system and the resulting performance
  n_gpu_layers=-1      # The number of layers to offload to GPU, if you have GPU acceleration available
)



# # Chat Completion API

# llm = Llama(model_path="./saiga_mistral_7b.Q4_K_M.gguf", chat_format="llama-2")  # Set chat_format according to the model you are using
# llm.create_chat_completion(
#     messages = [
#         {"role": "system", "content": "You are a story writing assistant."},
#         {
#             "role": "user",
#             "content": "Write a story about llamas."
#         }
#     ]
# )


: 

In [2]:
llm

In [3]:
from sentence_transformers import SentenceTransformer
import pandas as pd
from chromadb.utils import embedding_functions
from tqdm import tqdm
import chromadb
from chromadb.config import Settings

client = chromadb.Client()
# invest_qa_emb = client.create_collection(name=db_name, metadata={'hnsw:space': 'cosine'})
collection = client.create_collection(name="my_collection", embedding_function=embedding_functions.SentenceTransformerEmbeddingFunction(model_name="intfloat/multilingual-e5-small"), metadata={'hnsw:space': 'cosine'})

df = pd.read_excel('data/data.xlsx')
collection.add(
    documents=df.Answer.to_list(),
    metadatas=[{'question': q} for q in df.Theme.to_list()],
    ids=[str(i) for i in range(len(df))]
)


c:\Users\Ирина\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
embeddddd = SentenceTransformer("intfloat/multilingual-e5-small")
query = "Какие стратегии инвестирования есть?"
input_em = embeddddd.encode(sentences=f"query: {query}", batch_size=1, device = 'cuda').tolist()

results = collection.query(
    query_embeddings=[input_em],
    n_results=5
)

answers = dict(zip(results['documents'][0], results['distances'][0]))
for key, value in answers.items():
    answers[key] = 1 - value

In [5]:
answers

{'Есть три основные стратегии для начинающих — инвестирование на долгий срок в недооцененные компании, в стабильный доход или в компании роста.': 0.903084397315979,
 'Стратегия — личный план инвестора, в котором продумывается, из чего составлять портфель и как им управлять.': 0.9003602266311646,
 'Перед поиском идей надо определиться со стратегией инвестирования. Стратегия состоит из трех пунктов — цель, срок инвестирования и приемлемый уровень риска.': 0.8914626836776733,
 '1. Разработка стратегии: Разработайте инвестиционную стратегию, основанную на ваших целях, рисковых предпочтениях и тщательном анализе рынка. Придерживайтесь этой стратегии, даже если рынок сильно волатилен.\n2. Исследование и образование: Постоянно обновляйте свои знания о рынке и активах, в которые вы инвестируете. Чем лучше вы осведомлены, тем меньше вероятность влияния FOMO.\n3. Определение целей и рисков: Четко определите свои цели и готовность к риску. Это поможет вам принимать решения, основанные на ваших ли

In [9]:
# Simple inference example
system_message = f'Ты интеллектуальный помощник по инвестициям. Ты должен вежливо отвечать на вопросы пользователей, основываясь на предоставленных ответах. Тебе будут предостоставлены ответы на похожие вопросы из базы знаний в формате (ответ: коэфициент релевантности вопросу пользователя). Если среди предоставленных ответов есть информация которая отвечает на вопрос, то составь красивый и понятный ответ для пользователя, а если среди ответов нет информации, которая отвечает на вопрос, то переспроси пользователя(уточни что он имел ввиду), ни в коем случае не придумывай информацию!\
                    Вопрос пользователя "{query}".\n \
                    Ответы из базы знаний {answers}\
                    '
prompt = ''
output = llm(
  f"<|im_start|>system\n{system_message}<|im_end|>\n<|im_start|>user\n{prompt}<|im_end|>\n<|im_start|>assistant", # Prompt
  max_tokens=768,  # Generate up to 512 tokens
  stop=["</s>"],   # Example stop token - not necessarily correct for this specific model! Please check before using.
  echo=True        # Whether to echo the prompt
)


llama_print_timings:        load time =   35337.09 ms
llama_print_timings:      sample time =     116.12 ms /   726 runs   (    0.16 ms per token,  6252.10 tokens per second)
llama_print_timings: prompt eval time =   68173.23 ms /   967 tokens (   70.50 ms per token,    14.18 tokens per second)
llama_print_timings:        eval time =  256758.91 ms /   725 runs   (  354.15 ms per token,     2.82 tokens per second)
llama_print_timings:       total time =  327609.31 ms /  1692 tokens


In [1]:
output['choices'][0]['text']

NameError: name 'output' is not defined

In [3]:
from peft import PeftModel, PeftConfig
from transformers import AutoModelForCausalLM, AutoTokenizer, GenerationConfig
import torch

MODEL_NAME = "pirbis/Vikhr-7B-instruct_0.2-GGUF"
DEFAULT_MESSAGE_TEMPLATE = "<s>{role}\n{content}</s>\n"
DEFAULT_SYSTEM_PROMPT = "Ты — Вихрь, русскоязычный автоматический ассистент. Ты разговариваешь с людьми и помогаешь им."

class Conversation:
    def __init__(
        self,
        message_template=DEFAULT_MESSAGE_TEMPLATE,
        system_prompt=DEFAULT_SYSTEM_PROMPT,
    ):
        self.message_template = message_template
        self.messages = [{
            "role": "system",
            "content": system_prompt
        }]

    def add_user_message(self, message):
        self.messages.append({
            "role": "user",
            "content": message
        })

    def get_prompt(self, tokenizer):
        final_text = ""
        for message in self.messages:
            message_text = self.message_template.format(**message)
            final_text += message_text
        final_text += 'bot'
        return final_text.strip()


def generate(model, tokenizer, prompt, generation_config):
    data = tokenizer(prompt, return_tensors="pt")
    data = {k: v.to(model.device) for k, v in data.items()}
    output_ids = model.generate(
        **data,
        generation_config=generation_config
    )[0]
    output_ids = output_ids[len(data["input_ids"][0]):]
    output = tokenizer.decode(output_ids, skip_special_tokens=True)
    return output.strip()
    
config = PeftConfig.from_pretrained(MODEL_NAME)
model = AutoModelForCausalLM.from_pretrained(
    config.base_model_name_or_path,
    load_in_8bit=True,
    torch_dtype=torch.float16,
    device_map="auto"
)
#model = PeftModel.from_pretrained(    model,   MODEL_NAME,   torch_dtype=torch.float16)
model.eval()

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, use_fast=False)
generation_config = GenerationConfig.from_pretrained(MODEL_NAME)
print(generation_config)

inputs = ["Как тебя зовут?", "Кто такой Колмогоров?"]

for inp in inputs:
    conversation = Conversation()
    conversation.add_user_message(inp)
    prompt = conversation.get_prompt(tokenizer)

    output = generate(model, tokenizer, prompt, generation_config)
    print(inp)
    print(output)
    print('\n')


ValueError: Can't find 'adapter_config.json' at 'pirbis/Vikhr-7B-instruct_0.2-GGUF'